In [1]:
# 1.  Napisz program w Pythonie, który pobiera dane json i zwraca plik CSV z danymi.
# 2.  Wgraj wygenerowany plik CSV do Google Sheets.
# 3.  W nowym arkuszu stwórz tabelę przestawną, która przedstawia ile meteorów danej klasy spadło w danym roku.

In [10]:
import urllib.request
import json, csv
import pandas as pd
import os
from googleapiclient.http import MediaFileUpload
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

gauth = GoogleAuth()
gauth.LocalWebserverAuth() 
drive = GoogleDrive(gauth)



Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=505264712938-seano1obgad976pfvg351a5s4oc03647.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [37]:
def json_to_csv(link: str, name:str):
    data = dict()
    with urllib.request.urlopen(link) as url:
        data = json.loads(url.read().decode())
        #print(data)
    with open(f'{name}.json', 'w', ) as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    new_data = list()
    for d in data:
        new_d = dict()
        for key, value in d.items():
            if not isinstance(value, dict):
                new_d[key] = value
            else:
                for k, v in value.items():
                    new_d[key + "_" + k] = v
            new_data.append(new_d)
    df = pd.read_json('data.json')
    df = pd.DataFrame(new_data)
    df.to_csv(rf'{name}.csv', index = None)
    return df

In [39]:
df = json_to_csv("https://data.nasa.gov/resource/gh4g-9sfh.json", 'data')

In [48]:
pivot_table = df.groupby(["year", "recclass"])["id"].count().to_csv("pivot_table.csv")

In [50]:
def csv_to_google_sheets(file_path:str, file_name:str, google_folder:list = None):
    if not os.path.exists(file_path):
        print(f'{file_path} not found.')
        return
    try:
        file = drive.CreateFile({"mimeType":"text/csv", "parents":[{"kind": "drive#fileLink", "id": google_folder}]})
        file.SetContentFile(f'{file_name}.csv')
        file.Upload({'convert': True})
        print(file)
        return file
    except Exception as exp:
        print(exp)
        return    
    

In [52]:
csv_to_google_sheets('data.csv', 'data', '18xiAQC_UQxb0bhi3NNPDevBKlUztV5fl')
csv_to_google_sheets('pivot_table.csv', 'pivot_table', '18xiAQC_UQxb0bhi3NNPDevBKlUztV5fl')

GoogleDriveFile({'mimeType': 'application/vnd.google-apps.spreadsheet', 'parents': [{'kind': 'drive#parentReference', 'id': '18xiAQC_UQxb0bhi3NNPDevBKlUztV5fl', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1MxZAY5EBtsRVCM246PXZWEtbBJmiH0nNkzSzHnLvaGk/parents/18xiAQC_UQxb0bhi3NNPDevBKlUztV5fl', 'parentLink': 'https://www.googleapis.com/drive/v2/files/18xiAQC_UQxb0bhi3NNPDevBKlUztV5fl', 'isRoot': False}], 'title': 'data', 'kind': 'drive#file', 'id': '1MxZAY5EBtsRVCM246PXZWEtbBJmiH0nNkzSzHnLvaGk', 'etag': '"MTY1MTQzMDQxNDEzNA"', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1MxZAY5EBtsRVCM246PXZWEtbBJmiH0nNkzSzHnLvaGk', 'webContentLink': 'https://drive.google.com/uc?id=1MxZAY5EBtsRVCM246PXZWEtbBJmiH0nNkzSzHnLvaGk&export=download', 'alternateLink': 'https://docs.google.com/spreadsheets/d/1MxZAY5EBtsRVCM246PXZWEtbBJmiH0nNkzSzHnLvaGk/edit?usp=drivesdk', 'embedLink': 'https://docs.google.com/spreadsheets/d/1MxZAY5EBtsRVCM246PXZWEtbBJmiH0nNkzSzHnLvaGk/htmlembed?ouid=10776862

GoogleDriveFile({'mimeType': 'application/vnd.google-apps.spreadsheet', 'parents': [{'kind': 'drive#parentReference', 'id': '18xiAQC_UQxb0bhi3NNPDevBKlUztV5fl', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1FVdO9yRzxG4lUtj0KjeQ2XgjApXz7DZ-sSQm8Bd_oU0/parents/18xiAQC_UQxb0bhi3NNPDevBKlUztV5fl', 'parentLink': 'https://www.googleapis.com/drive/v2/files/18xiAQC_UQxb0bhi3NNPDevBKlUztV5fl', 'isRoot': False}], 'title': 'pivot_table', 'kind': 'drive#file', 'id': '1FVdO9yRzxG4lUtj0KjeQ2XgjApXz7DZ-sSQm8Bd_oU0', 'etag': '"MTY1MTQzMDQxNzYzMQ"', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1FVdO9yRzxG4lUtj0KjeQ2XgjApXz7DZ-sSQm8Bd_oU0', 'webContentLink': 'https://drive.google.com/uc?id=1FVdO9yRzxG4lUtj0KjeQ2XgjApXz7DZ-sSQm8Bd_oU0&export=download', 'alternateLink': 'https://docs.google.com/spreadsheets/d/1FVdO9yRzxG4lUtj0KjeQ2XgjApXz7DZ-sSQm8Bd_oU0/edit?usp=drivesdk', 'embedLink': 'https://docs.google.com/spreadsheets/d/1FVdO9yRzxG4lUtj0KjeQ2XgjApXz7DZ-sSQm8Bd_oU0/htmlembed?ouid=1